In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import time

connection = sqlite3.connect ('./dataset/database.sqlite')
data = pd.read_sql_query ('''SELECT*FROM Reviews where score != 3 LIMIT 5000''', connection)

def get_positiveNegative (x) :
    
    if x < 3 :
        return 0
    return 1
#     return x > 3 ? 1 : 0


filter_data = data['Score'].map (get_positiveNegative)
data['Score'] = filter_data

unique = pd.read_sql_query('''SELECT *,COUNT (*) FROM Reviews GROUP BY UserId HAVING COUNT(*) > 1''', connection)
unique.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,COUNT (*)
0,83318,B005ZBZLT4,#oc-R115TNMSPFT9I7,Breyton,2,3,2,1331510400,"""Green"" K-cup packaging sacrifices flavor",Overall its just OK when considering the price...,2
1,290948,B005HG9ESG,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,5,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u...",3
2,83657,B005ZBZLT4,#oc-R11DNU2NBKQ23Z,Kim Cieszykowski,0,2,1,1348531200,Terrible coffee,This coffee is horrible and unfortunately not ...,2
3,290774,B005HG9ESG,#oc-R11O5J5ZVQE25C,Penguin Chick,0,0,5,1346889600,Refreshing!,This will be the bottle that you grab from the...,3
4,181479,B007OSBEV0,#oc-R12KPBODL2B5ZD,Christopher P. Presta,5,8,1,1348617600,Not what I expected,I didnt like this coffee. Instead of telling y...,2


In [25]:
'''
data preprocessing
'''

sorted_data = data.sort_values('ProductId',axis=0,ascending=True,kind='quicksort',na_position='last') #na_position is the position of NaN data

non_redundant = sorted_data.drop_duplicates(subset={"UserId","Time","ProfileName","Text"},keep='first')
non_redundant = non_redundant[non_redundant['HelpfulnessNumerator']<=non_redundant['HelpfulnessDenominator']]



import re
from bs4 import BeautifulSoup


# regular expression
print (re.sub (r"http\S+","",non_redundant['Text'].values[0]))


print(non_redundant['Text'].values[0])
print(BeautifulSoup(non_redundant['Text'].values[0]))

text = BeautifulSoup(non_redundant['Text'].values[0])
text = text.get_text()

print(re.sub('[^A-Za-z0-9]+',' ',non_redundant['Text'].values[0]))
print(re.sub('\S*\d\S*','',non_redundant['Text'].values[0]))   # Removing Alfa-Numeric Words

def decontraction (phrase):

    phrase = re.sub (r"won't", "will not", phrase)
    phrase = re.sub (r"can\'t", "can not", phrase)

    phrase = re.sub (r"n\'t", " not", phrase)
    phrase = re.sub (r"\'re", " are", phrase)
    phrase = re.sub (r"\'s", " is", phrase)
    phrase = re.sub (r"\'d", " would", phrase)
    phrase = re.sub (r"\'ll", " will", phrase)
    phrase = re.sub (r"\'t", " not", phrase)
    phrase = re.sub (r"\'ve", " have", phrase)
    phrase = re.sub (r"\'m", " am", phrase)
    return phrase

print(non_redundant['Text'].values[900])
print(decontraction(non_redundant['Text'].values[900]))

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


<html><body><p>Why is this $[...] when the same product is available for $[...] here?<br/>http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br/><br/>The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.</p></body></html>


Why is this $[...] when the same product is available for $[...] here?http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDYThe Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but onl

In [13]:
def lower_case (sentence) :
    '''
    Convert capital to lower case
    '''
    sentence = ' '.join (e.lower () for e in sentence.split ())
    return sentence

print (lower_case (non_redundant['Text'].values[900]))

despite coming in an extremely large box, i found this to be great value. all the bags were preserved with a reasonable expiration date that is months away. i'd definitely recommend.
